In [1]:
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import pandas as pd

In [2]:
encoder = LabelEncoder()
pd.options.mode.chained_assignment = None

# Data 가지고 오기

In [3]:
genre_gn_all = pd.read_json('./Data/genre_gn_all.json', typ = 'series', encoding='utf-8')
song_meta = pd.read_json('./Data/song_meta.json', encoding='utf-8')
val= pd.read_json('./Data/val.json', encoding='utf-8', typ = 'frame')
train = pd.read_json('./Data/train.json', encoding='utf-8')
test = pd.read_json('./Data/test.json', encoding='utf-8')

# 데이터 형태 분석

In [4]:
# 장르코드 : gnr_code, 장르명 : gnr_name
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
genre_gn_all.head(3)

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80


In [5]:
song_meta.head(3)

,album_id,album_name,artist_id_basket,artist_name_basket,id,issue_date,song_gn_dtl_gnr_basket,song_gn_gnr_basket,song_name
0,2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[2727],[Various Artists],0,20140512,[GN0901],[GN0900],Feelings
1,376431,"Bach : Partitas Nos. 2, 3 & 4",[29966],[Murray Perahia],1,20080421,"[GN1601, GN1606]",[GN1600],"Bach : Partita No. 4 In D Major, BWV 828 - II...."
2,4698747,Hit,[3361],[Peter Gabriel],2,20180518,[GN0901],[GN0900],Solsbury Hill (Remastered 2002)


In [6]:
val.head(3)

,id,like_cnt,plylst_title,songs,tags,updt_date
0,118598,1675,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],2019-05-27 14:14:33.000
1,131447,1,앨리스테이블,[],[],2014-07-16 15:24:24.000
2,51464,62,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],2008-06-21 23:26:22.000


In [7]:
train.head(3)

,id,like_cnt,plylst_title,songs,tags,updt_date
0,61281,71,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],2013-12-19 18:36:19.000
1,10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",2014-12-02 16:19:42.000
2,76951,17,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",2017-08-28 07:09:34.000


In [8]:
test.head(3)

,id,like_cnt,plylst_title,songs,tags,updt_date
0,70107,6,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],2012-09-29 01:57:26.000
1,7461,0,,"[196298, 269984, 267805, 175867, 529244, 63825...","[나만의Best3, 인디아티스트들의추천음악]",2019-12-17 14:06:45.000
2,90348,21,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브],2015-05-23 10:44:48.000


# Info 보기

In [9]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
album_id                  707989 non-null int64
album_name                707985 non-null object
artist_id_basket          707989 non-null object
artist_name_basket        707989 non-null object
id                        707989 non-null int64
issue_date                707989 non-null int64
song_gn_dtl_gnr_basket    707989 non-null object
song_gn_gnr_basket        707989 non-null object
song_name                 707989 non-null object
dtypes: int64(3), object(6)
memory usage: 48.6+ MB


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
id              115071 non-null int64
like_cnt        115071 non-null int64
plylst_title    115071 non-null object
songs           115071 non-null object
tags            115071 non-null object
updt_date       115071 non-null object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB


In [11]:
# 곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# unnest 객체 제거
del song_gnr_map_unnest

In [12]:
song_gnr_map.head(3)

,song_id,gnr_code
0,0,GN0900
1,1,GN1600
2,2,GN0900


# 데이터 전처리

# 장르 처리

In [13]:
# 상세 장르 코드
song_meta.T[6:-2].T.head(3)

,song_gn_dtl_gnr_basket
0,[GN0901]
1,"[GN1601, GN1606]"
2,[GN0901]


In [14]:
encoder.fit(genre_gn_all.gnr_code)
tmp = pd.DataFrame(encoder.transform(genre_gn_all.gnr_code))

In [15]:
tmp = tmp.rename(columns={
    '0': 'gnr_code'
})

gnr_code = genre_gn_all['gnr_code']

genre_gn_all['gnrOneHot'] = tmp

genre_gn_all.head(3)

,gnr_code,gnr_name,gnrOneHot
0,GN0100,발라드,0
1,GN0101,세부장르전체,1
2,GN0102,'80,2


# Tag 처리

In [16]:
from sklearn import preprocessing

# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

tags = plylst_tag_map.tags

tags.T.tail(3)


encoder.fit(tags)
tmp = pd.DataFrame(encoder.transform(plylst_tag_map.tags))

plylst_tag_map['tagEncoding'] = tmp

del tmp

plylst_tag_map.head(3)

,id,tags,tagEncoding
0,61281,락,10503
1,10532,추억,24659
2,10532,회상,28557


In [17]:
df = pd.DataFrame({"tags":plylst_tag_map.tags, "tagEncoding":plylst_tag_map.tagEncoding})

name_idx_map = { r['tags']:r['tagEncoding'] for _, r in df.iterrows() }

data = pd.Series(train.tags)

train['tagsEncoding'] = data.apply(lambda e: [ name_idx_map[k] for k in e ])

del data, name_idx_map
train.head()

,id,like_cnt,plylst_title,songs,tags,updt_date,tagsEncoding
0,61281,71,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],2013-12-19 18:36:19.000,[10503]
1,10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",2014-12-02 16:19:42.000,"[24659, 28557]"
2,76951,17,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",2017-08-28 07:09:34.000,"[7398, 22262]"
3,147456,33,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",2019-12-05 15:15:18.000,"[19178, 8617, 25157, 13828, 10191, 25625, 5797..."
4,27616,9,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],2011-10-25 13:54:56.000,[9247]


In [18]:
train['tags'] = train['tags'].fillna('')
train.head(2)

,id,like_cnt,plylst_title,songs,tags,updt_date,tagsEncoding
0,61281,71,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],2013-12-19 18:36:19.000,[10503]
1,10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",2014-12-02 16:19:42.000,"[24659, 28557]"


In [27]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
    return doc

tfidf = CountVectorizer(
    tokenizer=dummy,
    preprocessor=dummy,
) 
tfidf_matrix = tfidf.fit_transform(train.tags)
# overview에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(115071, 29160)


In [25]:
tfidf.get_feature_names()

['',
 '00',
 '007',
 '007시리즈',
 '00s',
 '00년',
 '00년대',
 '00년대노래',
 '00년대발라드',
 '00년도노래',
 '00년생',
 '01',
 '0129',
 '015B',
 '02',
 '03',
 '03월',
 '04',
 '0416',
 '0421',
 '04_29',
 '05',
 '0515',
 '06',
 '07',
 '0802',
 '08월',
 '0930',
 '09월',
 '0channel',
 '0개국어',
 '0살',
 '0시',
 '1',
 '10',
 '100',
 '100000',
 '1000곡',
 '1000만관객동원',
 '1000칼로리',
 '100곡',
 '100번',
 '100선',
 '100억짜리피쳐링',
 '100위',
 '100주년',
 '101',
 '10cm',
 '10곡',
 '10년간의_기록',
 '10년대',
 '10년전',
 '10대',
 '10대가10대에게',
 '10대감성',
 '10대들의우상',
 '10대의영원할것같았던순수함',
 '10만',
 '10만개',
 '10분전',
 '10살',
 '10센치',
 '10월',
 '10월29일',
 '10월4주차',
 '10월_1주차',
 '10월_2주차',
 '10월_3주차',
 '10월_4주차',
 '10월마지막힙합',
 '10월음악',
 '10월힙합',
 '10주',
 '10주년',
 '10주년축하해',
 '10초',
 '10화',
 '11',
 '110bpm',
 '1124',
 '113',
 '11곡',
 '11시',
 '11월',
 '11월11일',
 '11월1일',
 '11월1주차',
 '11월20일',
 '11월3주차',
 '11월4주차',
 '11월_2주차',
 '11월_3주차',
 '11월_컴백',
 '11월넷째주',
 '11월둘째주',
 '11일',
 '11화',
 '12',
 '1218',
 '1222',
 '1224',
 '1225',
 '123',
 '1248',
 '127',
 '128',


In [30]:
# from sklearn.metrics.pairwise import linear_kernel
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# indices = pd.Series(data.index, index=data['tags']).drop_duplicates()
# print(indices.head())

In [34]:
# from sklearn.metrics.pairwise import cosine_similarity

# genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix, batch_size = 40)
# print(genre_sim.shape)
# print(genre_sim[:2])

In [57]:
# tmp = np.array(train.tags)
# train.songs.shape

# from tensorflow.keras.layers import Input, Dense
# from tensorflow.keras.models import Model

# # MNIST input 28 rows * 28 columns = 784 pixels
# input_img = Input(shape=(7,))
# # encoder
# encoder1 = Dense(128, activation='sigmoid')(input_img)
# encoder2 = Dense(3, activation='sigmoid')(encoder1)
# # decoder
# decoder1 = Dense(128, activation='sigmoid')(encoder2)
# decoder2 = Dense(7, activation='sigmoid')(decoder1)

# # this model maps an input to its reconstruction
# autoencoder = Model(inputs=input_img, outputs=decoder2)

# autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [58]:
# autoencoder.fit(train, train,
#                 epochs=5,
#                 batch_size=32,
#                 shuffle=True,
#                 validation_data=(test.tags, test.tags))

In [ ]:
def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    se_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

# def rec

def cos_sim(A, B):
       return np.dot(A, B)/(norm(A)*norm(B))

In [ ]:
# print(cos_sim(genre_mat, genre_mat))

In [62]:
import keras

keras.losses.CosineSimilarity(
    axis=-1, reduction=losses_utils.ReductionV2.AUTO, name='cosine_similarity'
)

cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)

model = tf.keras.Model(tfidf_matrix, tfidf_matrix)
model.compile('sgd', loss=tf.keras.losses.CosineSimilarity(axis=1))

Using TensorFlow backend.


AttributeError: module 'keras.losses' has no attribute 'CosineSimilarity'

# 학습

In [ ]:
df = pd.DataFrame({"name":["발", "팝", "팔"], "onehot":[0,1,2]})
df

In [ ]:
name_idx_map = { r['name']:r['onehot'] for _, r in df.iterrows() }
name_idx_map

In [ ]:
data = pd.Series([ ['발'], ['팝', '팔']])
data.dtype

In [ ]:
data.apply(lambda e: [ name_idx_map[k] for k in e ])